# Параметрические критерии

## Одновыборочный критерий Стьюдента

In [1]:
import numpy as np
import pandas as pd
import sklearn

from scipy import stats
from statsmodels.stats.weightstats import _tconfint_generic, _zconfint_generic
from statsmodels.stats.proportion import *

from sklearn import model_selection, linear_model, ensemble, metrics

### Задача 1

Средний вес детей при рождении составляет 3.3 кг, у женщин, живущих за чертой бедности - 2.8 кг.<br><br>
25 женщин, живущих за чертой бедности, участвовали в экспериментальной программе по ведению беременности. Средний вес их детей при рождении составил 3075 г. со стандартным отклонением 500 г.<br><br>
Необходимо понять, эффективна ли программа.

Нулевая гипотеза $H_0$: программа не эффективна, то есть $\mu=\mu_0=2800$.<br><br>
В данном случае стандартное отклонение оценено по выборке, поэтому нужно для проверки гипотезы нужно применить одновыборочный t-критерий Стьюдента.

Пусть альтернативная гипотеза $H_1$ будет двусторонней: $\mu\neq\mu_0$.<br><br>
Тогда достигаемый уровень значимости вычисляется по следующей формуле:
<br>
<div align='center'> $p=P(x<t)+P(x>t)=F_{St(n-1)}(t)+(1-F_{St(n-1)}(t))=2*(1-F_{St(n-1)}(|t|))$. </div>

In [2]:
n=25
X_mean=3075
mu_0=2800
S=500
t=(X_mean-mu_0)/(S/np.sqrt(n))
print('Достигаемый уровень значимости:', 2*(1-stats.t.cdf(t, df=n-1)))

Достигаемый уровень значимости: 0.011147829812680365


Полученное p-value меньше, чем 0.05, следовательно $H_0$ отвергается в пользу $H_1$ - эксперимент оказывает эффект на вес новорожденных.

In [5]:
edges=_tconfint_generic(X_mean-mu_0,S/np.sqrt(n),n-1, 0.05, 'two-sided')
print('95% доверительный интервал для прироста [{:.4f}, {:.4f}]:'.format(edges[0],edges[1]))

95% доверительный интервал для прироста [68.6101, 481.3899]:


Пусть альтернативная гипотеза $H_1$ будет односторонней: $\mu>\mu_0$.<br><br>
Тогда достигаемый уровень значимости вычисляется по следующей формуле:
<br>
<div align='center'> $p=P(x>t)=1-F_{St(n-1)}(t)$. </div>

In [6]:
print('Достигаемый уровень значимости:', 1-stats.t.cdf(t, df=n-1))

Достигаемый уровень значимости: 0.005573914906340183


Полученное p-value в данном случае также меньше, чем 0.05, следовательно $H_0$ отвергается в пользу $H_1$ - эксперимент оказывает эффект на вес новорожденных в большую сторону.

In [7]:
edges=_tconfint_generic(X_mean,S/np.sqrt(n),n-1, 0.05, 'larger')
print('95% доверительный интервал для прироста [{:.4f}, {:.4f}]:'.format(edges[0]-mu_0,edges[1]-mu_0))

95% доверительный интервал для прироста [103.9118, inf]:


### Задача 2

Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?

$H_0$: средний уровень кальция в гватемальской популяции не отличается от 9.5 мг/дл.<br>
$H_1$: отличается.

Так как в данном случае известно стандартное отклонение, то в качестве критерия будет использоваться z-критерий Стьюдента.

In [8]:
n=160
X_mean=9.57
mu_0=9.5
sigma=0.4
z=(X_mean-mu_0)/(sigma/np.sqrt(n))
print('Достигаемый уровень значимости: %.4f'%(2*(1-stats.norm.cdf(z))))

Достигаемый уровень значимости: 0.0269


_p-value_ меньше уровня значимости 0.05, следовательно, нулевая гипотеза может быть отвергнута в пользу альтернативной.

In [11]:
edges=_zconfint_generic(X_mean,sigma/np.sqrt(n),0.05, 'two-sided')
print('95% доверительный интервал для прироста [{:.4f}, {:.4f}]:'.format(edges[0]-mu_0,edges[1]-mu_0))

95% доверительный интервал для прироста [0.0080, 0.1320]:


Видим, что в 95 процентах случаев прирост уровня кальция оказывается значительно меньше стандартного отклонения. Поэтому можно сделать вывод, что такое различие кальция в крови является статистически значимым, но не практически.

### Задача 3

Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.<br>
Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. 

$H_0$: подготовительные курсы не эффективны - средний балл после них не отличается от 525 ($\overline X=525$).<br>
$H_1$: средний балл после подготовительных курсов повышается ($\overline X>525$).

Так как в данном случае известно стандартное отклонение, то в качестве критерия будет использоваться z-критерий Стьюдента.

In [12]:
n=100
X_mean=541.4
mu_0=525
sigma=100
z=(X_mean-mu_0)/(sigma/np.sqrt(n))
print('Достигаемый уровень значимости: %.4f'%(1-stats.norm.cdf(z)))

Достигаемый уровень значимости: 0.0505


На уровне значимости 0.05 нулевая гипотеза не может быть отвергнута, значит, по этим данным курсы нельзя считать эффективными.

## Двухвыборочный критерий Стьюдента для связанных выборок

### Задача 4

Имеются данные о стоимости и размерах 53940 бриллиантов.<br><br>
Необходимо обучить две регрессионные модели: линейную и случайный лес, а затем сравнить их качество. 

In [13]:
data=pd.read_csv('data/diamonds.txt', header=0, sep='\t')

In [14]:
data.head()

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


In [15]:
y=data['price'].values
X=data.drop('price', axis=1)

In [16]:
X_train, X_test, y_train, y_test=model_selection.train_test_split(X,y,test_size=0.25,random_state=1)

Создадим две регрессионные модели: линейную и случайный лес.

In [17]:
linregr=linear_model.LinearRegression()
linregr.fit(X_train, y_train)
predictions_linregr=linregr.predict(X_test)
errors_linregr=np.fabs(y_test-predictions_linregr)

In [18]:
forest=ensemble.RandomForestRegressor(n_estimators=10,random_state=1)
forest.fit(X_train, y_train)
predictions_forest=forest.predict(X_test)
errors_forest=np.fabs(y_test-predictions_forest)

In [19]:
errors_linregr.mean()

890.3764004285612

In [20]:
errors_forest.mean()

802.9205172724115

Точечная оценка средних по массивам ошибок говорит о том, что случайный лес лучше справляется с задачей предсказания цены. 

Для более достоверного результата произведем оценку качества предсказаний с помощью двухвыборочного критерия Стьюдента.

Получается, что перед нами есть две связанные выборки. Для вычисления соответсвующего критерия модуль _stats_ библиотеки  _scipy_ предоставляет функциюкцию _ttest_rel()_.

$H_0$: средние значения ошибок построенных моделей не отличаются (качество моделей одинаковое).<br>
$H_1$: качество моделей разное.

In [21]:
stats.ttest_rel(errors_linregr, errors_forest)

Ttest_relResult(statistic=13.017729783878696, pvalue=1.655174575138418e-38)

Достигаемый уровень значимости получился очень маленьким, поэтому нулевая гипотеза может быть смело отвергнута на уровне значимости $\alpha=0.05$.

Чтобы дополнительно удостовериться в полученном выводе, произведем интервальную оценку разности средних значений абсолютных отклонений двух моделей.

In [22]:
errors_linregr=y_test-predictions_linregr
errors_forest=y_test-predictions_forest
print("95%%  доверительный интервал для разности средних абсолютных отклонений: [%f, %f]" \
      % DescrStatsW(errors_linregr - errors_forest).tconfint_mean())

95%  доверительный интервал для разности средних абсолютных отклонений: [13.230427, 44.252558]


Доверительный интервал не содержит ноль - средняя ошибка линейной модели действительно больше, чем средняя ошибка случайного леса.

## Z-критерий для доли

### Задача 5

На одном из этапов выбора присяжных было отобрано 300 кандидатов, среди которых оказалось только 90 женщин. Был ли отбор беспристрастным? 

$H_0$: отбор был беспристрастным, то есть вероятности женщин и мужчин стать присяжными одинаковы и равны $p_0=\frac{1}{2}$.
<br>
$H_1$: отбор не был беспристрастным.

Перед нами выборка, подчиняющаяся биноминальному закону распределения. Точечная оценка вероятности женщин попасть в группу присяжных равна отношению числа женщин к числу всех кандидатов:
$$\hat{p}=\frac{90}{300}$$
Так как выборка биноминальная, то величину $\hat{p}$ можно интерпретировать как точечную оценку матожидания. Матожидание, с которым мы будем сравнивать данное значение, равно $p_0$. Дисперсия: $\sqrt{\frac{p_0*(1-p_0)}{n}}$. Таким образом, мы будем вычислять z-статистику на основе критерия множетилей Лагранжа: $$z=\frac{\hat p - p_0}{\sqrt{\frac{p_0*(1-p_0)}{n}}}$$

In [14]:
n=300
n_women=90
p=n_women/n
p_0=0.5
std=np.sqrt(p_0*(1-p_0)/n)
z=(p-p_0)/std
print('z-критерий: %.4f, p-value: %g'%(z,2*(1-stats.norm.cdf(np.abs(z)))))

z-критерий: -6.9282, p-value: 4.26215e-12


Достигаемый уровень значимости очень мал, что позволяет уверенно отклонить нулевую гипотезу.

Существует библиотечная функция _proportiions_ztest_, которая использует критерий Вальда для вычисления z-статистки, который является менее точным по сравнению с критерием множителей Лагранжа. 

In [16]:
z,p=proportions_ztest(n_women,n,value=0.5)
print('z-критерий: %.4f, p-value: %g'%(z,p))

z-критерий: -7.5593, p-value: 4.05277e-14


In [24]:
print('95%% доверительный интервал для истинного значения вероятности женщин оказаться в числе присяжных: [%.4f, %.4f].'\
      %proportion_confint(n_women, n, alpha=0.05, method='normal'))

95% доверительный интервал для истинного значения вероятности женщин оказаться в числе присяжных: [0.2481, 0.3519].


## Z-критерий для двух связанных выборок

### Задача 6

Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. В файле находятся данные по измеренным признакам.<br>
Необходимо составить два классификатора, один из которых будет принимать решение на основе признаков $X_1$, $X_2$, $X_3$, а другой - на основе $X_4$, $X_5$, $X_6$. Затем определить, различаются ли доли правильных ответов этих моделей, и если да, то как.

In [25]:
data=pd.read_csv('data/banknotes.txt', header=0, sep='\t')
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [26]:
y=data.real.values
X=data.drop('real',axis=1)

In [27]:
X_train, X_test, y_train, y_test=model_selection.train_test_split(X, y, test_size=50, random_state=1)

In [83]:
model1=linear_model.LogisticRegression(multi_class='ovr', n_jobs=1, solver='liblinear').fit(X_train[['X1','X2','X3']],y_train)
predictions1=model1.predict(X_test[['X1','X2','X3']])
model2=linear_model.LogisticRegression(multi_class='ovr', n_jobs=1, solver='liblinear').fit(X_train[['X4','X5','X6']],y_train)
predictions2=model2.predict(X_test[['X4','X5','X6']])

In [84]:
p1=metrics.accuracy_score(y_test, predictions1)
p2=metrics.accuracy_score(y_test, predictions2)
print('accuracy первой модели: %f'%p1)
print('accuracy второй модели: %f'%p2)

accuracy первой модели: 0.800000
accuracy второй модели: 0.980000


Точечные оценки говорят, что вторая модель является более эффективной, однако проверим данный вывод на статистическом уровне.

$H_0$: доли ошибочных ответов, полученные с помощью двух моделей, не отличаются.<br>
$H_1$: отличаются.

Предсказания выполняются для одних и тех же объектов, поэтому выборки _predictions_ являются связанными.

In [85]:
errors1=np.abs(predictions1-y_test)
errors2=np.abs(predictions2-y_test)
errors2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int64)

In [113]:
def zstat_prop_dif_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n=len(sample1)
    a = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    b = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    z=(a-b)/np.sqrt(a+b-((a-b)**2)/n)
    return z


In [114]:
def pvalue(stat,h1):
    if h1=='two-sided':
        return 2*(1-stats.norm.cdf(np.fabs(stat)))
    elif h1=='greater':
        return 1-stats.norm.cdf(stat)
    elif h1=='less':
        return stats.norm.cdf(stat)

In [116]:
print('Достигаемый уровень значимости для разности долей: %f'\
      %pvalue(zstat_prop_dif_rel(errors1, errors2),'two-sided'))

Достигаемый уровень значимости для разности долей: 0.003297


Видно, что на уровне значимости 0.05 нулевая гипотеза может быть отвергнута в пользу двусторонней альтернативы.

In [117]:
print('Достигаемый уровень значимости для разности долей: %f'\
      %pvalue(zstat_prop_dif_rel(errors1, errors2),'greater'))

Достигаемый уровень значимости для разности долей: 0.001648


При этом, p-value для односторонней альтернативы получается еще меньше, что дает нам еще большую уверенность в лучшей работе второй модели.

In [140]:
def prop_diff_confint(sample1, sample2,alpha):
    z=stats.norm.ppf(1-alpha/2)
    n=len(sample1)
    sample = list(zip(sample1, sample2))
    a = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    b = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    upper=(a-b)/n+z*np.sqrt((a+b)/n**2-((a-b)**2)/n**3)
    lower=(a-b)/n-z*np.sqrt((a+b)/n**2-((a-b)**2)/(n**3))
    return (lower,upper)

In [141]:
print('95%% доверительный интервал для разности долей: [%.4f, %.4f].'%prop_diff_confint(errors1, errors2, 0.05))

95% доверительный интервал для разности долей: [0.0599, 0.3001].


Интервал лежит справа от нуля, что позволяет сказать, что доля ошибок первого классификатора больше доли ошибок второго.